In [ ]:
!pip install yfinance

In [ ]:
import sqlite3
import pandas as pd
import yfinance as yf

In [ ]:
balance_sheet = []
income_statement = []
cfs = []
years = []
profitability_score = 0
leverage_score = 0
operating_efficiency_score = 0
pe_ratio = 0
sector = ''
industry = ''
short_name = ''
market_capital = 0
sigma_analytics = 'drive/MyDrive/Colab Notebooks/sigma-analytics/'

In [ ]:
columns = ['Ticker', 'Short Name', 'Sector', 'Industry', 'Market Capital', 'PE ratio', 'Profitability', 'Leverage', 'Operating eficiency', 'Piotroski F-score']
summary = pd.DataFrame(columns = columns)
large_cap_summary = pd.DataFrame(columns = columns)
mid_cap_summary = pd.DataFrame(columns = columns)
small_cap_summary = pd.DataFrame(columns = columns)
conn = sqlite3.connect(sigma_analytics+'F-Score_Top_100_100_50-2021.db')

In [ ]:
tickers = []
tickers.append(pd.read_csv("https://www1.nseindia.com/content/indices/ind_nifty100list.csv")["Symbol"].tolist())
tickers.append(pd.read_csv("https://www1.nseindia.com/content/indices/ind_niftymidcap100list.csv")["Symbol"].tolist())
tickers.append(pd.read_csv("https://www1.nseindia.com/content/indices/ind_niftysmallcap50list.csv")["Symbol"].tolist())

In [ ]:
def get_data(ticker: yf.Ticker):
    global balance_sheet
    global income_statement
    global cfs
    global years
    balance_sheet = ticker.get_balance_sheet()
    income_statement = ticker.get_financials()
    cfs = ticker.get_cashflow()
    years = balance_sheet.columns  

def info(ticker: yf.Ticker):
    global sector
    global industry
    global market_capital
    global short_name
    global pe_ratio
    sector = ticker.get_info()['sector']
    industry = ticker.get_info()['industry']
    market_capital = ticker.get_info()['marketCap']
    short_name = ticker.get_info()['shortName']
    try:
      pe_ratio = ticker.get_info()['trailingPE']
    except:
      pe_ratio = 0  

def profitability():
    global profitability_score
    #Scores #1 and 2 - net income
    net_income = income_statement[years[0]]['Net Income']
    net_income_py = income_statement[years[1]]['Net Income']
    ni_score = 1 if net_income > 0 else 0
    ni_score_2 = 1 if net_income > net_income_py else 0

    #Score #3 - operating cash flow
    op_cf = cfs[years[0]]['Total Cash From Operating Activities']
    op_cf_score = 1 if op_cf > 0 else 0

    #Score #4 - change in RoA
    avg_assets = (balance_sheet[years[0]]['Total Assets']
                    + balance_sheet[years[1]]['Total Assets']) / 2
    avg_assets_py = (balance_sheet[years[1]]['Total Assets']
                    + balance_sheet[years[2]]['Total Assets']) / 2
    RoA = net_income / avg_assets
    RoA_py = net_income_py / avg_assets_py
    RoA_score = 1 if RoA > RoA_py else 0

    #Score #5 - Accruals
    total_assets = balance_sheet[years[0]]['Total Assets']
    accruals = op_cf / total_assets - RoA
    ac_score = 1 if accruals > 0 else 0

    profitability_score = ni_score + ni_score_2 + op_cf_score + RoA_score + ac_score

def leverage():
    global leverage_score
    #Score #6 - long-term debt ratio
    try:
        lt_debt = balance_sheet[years[0]]['Total Liab'] - balance_sheet[years[0]]['Total Current Liabilities']
        total_assets = balance_sheet[years[0]]['Total Assets']
        debt_ratio = lt_debt / total_assets
        debt_ratio_score = 1 if debt_ratio < 0.4 else 0
    except:
        debt_ratio_score = 1

    #Score #7 - Current ratio
    current_assets = balance_sheet[years[0]]['Total Current Assets']
    current_liab = balance_sheet[years[0]]['Total Current Liabilities']
    current_ratio = current_assets / current_liab
    current_ratio_score = 1 if current_ratio > 1 else 0

    leverage_score = debt_ratio_score + current_ratio_score

def operating_efficiency():
    global operating_efficiency_score
    #Score #8 - Gross margin
    gp = income_statement[years[0]]['Gross Profit']
    gp_py = income_statement[years[1]]['Gross Profit']
    revenue = income_statement[years[0]]['Total Revenue']
    revenue_py = income_statement[years[1]]['Total Revenue']
    gm = gp / revenue
    gm_py = gp_py / revenue_py
    gm_score = 1 if gm > gm_py else 0
    
    #Score #9 - Asset turnover
    avg_assets = (balance_sheet[years[0]]['Total Assets']
                    + balance_sheet[years[1]]['Total Assets']) / 2
    avg_assets_py = (balance_sheet[years[1]]['Total Assets']
                    + balance_sheet[years[2]]['Total Assets']) / 2

    at = revenue / avg_assets #at = asset turnover
    at_py = revenue_py / avg_assets_py
    at_score = 1 if at > at_py else 0

    operating_efficiency_score = gm_score + at_score

In [ ]:
for index, items in enumerate(tickers):
  for item in items:
    try:
        ticker = yf.Ticker(item+'.NS')
        get_data(ticker)
        info(ticker)
        profitability()
        leverage()
        operating_efficiency()
        new_row = {'Ticker': item,
                   'Short Name': short_name,
                   'Sector': sector,
                   'Industry': industry,
                   'Market Capital': market_capital,
                   'PE ratio': pe_ratio,
                   'Profitability': profitability_score,
                   'Leverage': leverage_score,
                   'Operating eficiency': operating_efficiency_score,
                   'Piotroski F-score': profitability_score+leverage_score+operating_efficiency_score}
        if index == 0:
          large_cap_summary = large_cap_summary.append(new_row, ignore_index = True)
        elif index == 1:
          mid_cap_summary = mid_cap_summary.append(new_row, ignore_index = True)
        else:
          small_cap_summary = small_cap_summary.append(new_row, ignore_index = True)
        summary = summary.append(new_row, ignore_index = True)
        print(item + ' added.')
    except:
        print(item + ': Something went wrong.')

ACC added.
ADANIENT added.
ADANIGREEN added.
ADANIPORTS added.
ADANITRANS added.
AMBUJACEM added.
APOLLOHOSP added.
ASIANPAINT added.
DMART added.
AXISBANK added.
BAJAJ-AUTO added.
BAJFINANCE added.
BAJAJFINSV added.
BAJAJHLDNG added.
BANDHANBNK added.
BANKBARODA added.
BERGEPAINT added.
BPCL added.
BHARTIARTL added.
BIOCON added.
BOSCHLTD added.
BRITANNIA added.
CHOLAFIN added.
CIPLA added.
COALINDIA added.
COLPAL added.
DLF added.
DABUR added.
DIVISLAB added.
DRREDDY added.
EICHERMOT added.
NYKAA added.
GAIL added.
GLAND added.
GODREJCP added.
GRASIM added.
HCLTECH added.
HDFCAMC added.
HDFCBANK added.
HDFCLIFE added.
HAVELLS added.
HEROMOTOCO added.
HINDALCO added.
HINDUNILVR added.
HDFC added.
ICICIBANK added.
ICICIGI added.
ICICIPRULI added.
ITC added.
IOC added.
INDUSTOWER added.
INDUSINDBK added.
NAUKRI added.
INFY added.
INDIGO added.
JSWSTEEL added.
JUBLFOOD added.
KOTAKBANK added.
LTI added.
LT added.
LUPIN added.
M&M added.
MARICO added.
MARUTI added.
MINDTREE added.
MUTHOOT

In [ ]:
summary.to_sql('all_cap', conn, if_exists='replace', index=False)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:2882: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


In [ ]:
large_cap_summary.to_sql('large_cap', conn, if_exists='replace', index=False)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:2882: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


In [ ]:
mid_cap_summary.to_sql('mid_cap', conn, if_exists='replace', index=False)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:2882: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


In [ ]:
small_cap_summary.to_sql('small_cap', conn, if_exists='replace', index=False)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:2882: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


In [ ]:
sql = '''select * from all_cap'''
pd.read_sql(sql, conn)

,Ticker,Short Name,Sector,Industry,Market Capital,PE ratio,Profitability,Leverage,Operating eficiency,Piotroski F-score
0,ACC,ACC LIMITED,Basic Materials,Building Materials,414755749888,22.323124,5,2,1,8
1,ADANIENT,ADANI ENTERPRISES,Energy,Thermal Coal,2418977275904,342.700200,3,1,1,5
2,ADANIGREEN,ADANI GREEN ENERGY,Utilities,Utilities—Renewable,4480509673472,2730.505100,5,0,0,5
3,ADANIPORTS,ADANI PORT SPECIAL,Industrials,Marine Shipping,1775447048192,35.215990,5,1,1,7
4,ADANITRANS,ADANI TRANSMISSION,Utilities,Utilities—Regulated Electric,2959313993728,306.848000,5,0,1,6
...,...,...,...,...,...,...,...,...,...,...
244,ROUTE,ROUTE MOBILE LIMITED,Communication Services,Telecom Services,98660081664,59.050255,5,2,1,8
245,STLTECH,STERLITE TECHNOLOGIES LTD,Technology,Communication Equipment,85458092032,41.143240,3,1,0,4
246,UTIAMC,UTI Asset Mngmt Co Ltd,Financial Services,Asset Management,121871040512,19.885656,4,2,2,8
247,WELSPUNIND,WELSPUN INDIA,Consumer Cyclical,Textile Manufacturing,94952374272,14.113673,5,2,1,8


In [ ]:
sql = '''select * from large_cap'''
pd.read_sql(sql, conn)

,Ticker,Short Name,Sector,Industry,Market Capital,PE ratio,Profitability,Leverage,Operating eficiency,Piotroski F-score
0,ACC,ACC LIMITED,Basic Materials,Building Materials,414755749888,22.323124,5,2,1,8
1,ADANIENT,ADANI ENTERPRISES,Energy,Thermal Coal,2418977275904,342.700200,3,1,1,5
2,ADANIGREEN,ADANI GREEN ENERGY,Utilities,Utilities—Renewable,4480509673472,2730.505100,5,0,0,5
3,ADANIPORTS,ADANI PORT SPECIAL,Industrials,Marine Shipping,1775447048192,35.215990,5,1,1,7
4,ADANITRANS,ADANI TRANSMISSION,Utilities,Utilities—Regulated Electric,2959313993728,306.848000,5,0,1,6
...,...,...,...,...,...,...,...,...,...,...
95,MCDOWELL-N,UNITED SPIRITS,Consumer Defensive,Beverages—Wineries & Distilleries,649723772928,74.211360,3,2,0,5
96,VEDL,VEDANTA LIMITED,Basic Materials,Other Industrial Metals & Mining,1582783660032,8.175412,5,1,2,8
97,WIPRO,WIPRO LTD,Technology,Information Technology Services,3057370267648,25.304312,5,2,2,9
98,ZOMATO,Zomato Limited,Consumer Cyclical,Internet Retail,660454899712,0.000000,3,2,0,5


In [ ]:
sql = '''select * from mid_cap'''
pd.read_sql(sql, conn)

,Ticker,Short Name,Sector,Industry,Market Capital,PE ratio,Profitability,Leverage,Operating eficiency,Piotroski F-score
0,ABB,ABB INDIA LTD,Industrials,Specialty Industrial Machinery,472247435264,90.793070,5,2,2,9
1,AUBANK,AU SMALL FINANCE B,Financial Services,Banks—Regional,441759956992,46.454950,3,1,1,5
2,AARTIIND,AARTI IND LTD,Basic Materials,Specialty Chemicals,345502416896,27.368300,3,1,1,5
3,ABBOTINDIA,ABBOTT INDIA LTD,Healthcare,Drug Manufacturers—General,378344865792,51.144543,5,2,1,8
4,ATGL,ADANI TOTAL GAS LIMITED,Utilities,Utilities—Regulated Gas,2750459936768,481.025240,4,1,1,6
...,...,...,...,...,...,...,...,...,...,...
95,IDEA,VODAFONE IDEA LTD,Communication Services,Telecom Services,334035517440,0.000000,4,0,1,5
96,VOLTAS,VOLTAS LTD,Industrials,Specialty Industrial Machinery,431573303296,77.232350,4,2,0,6
97,WHIRLPOOL,WHIRLPOOL OF INDIA,Consumer Cyclical,"Furnishings, Fixtures & Appliances",210772459520,34.408980,3,2,0,5
98,YESBANK,YES BANK,Financial Services,Banks—Regional,364550258688,0.000000,2,1,1,4


In [ ]:
sql = '''select * from small_cap'''
pd.read_sql(sql, conn)

,Ticker,Short Name,Sector,Industry,Market Capital,PE ratio,Profitability,Leverage,Operating eficiency,Piotroski F-score
0,ALOKINDS,Alok Industries Limited,Consumer Cyclical,Textile Manufacturing,137537175552,0,2,0,2,4
1,AMARAJABAT,AMARA RAJA BATTERI,Industrials,Electrical Equipment & Parts,97311596544,16.135155,3,2,0,5
2,AMBER,AMBER ENTERPRISES,Consumer Cyclical,"Furnishings, Fixtures & Appliances",120768331776,94.880486,3,2,1,6
3,ANGELONE,ANGEL ONE LIMITED,Financial Services,Capital Markets,139787911168,26.940987,3,2,2,7
4,ANURAS,Anupam Rasayan India Ltd,Basic Materials,Specialty Chemicals,87404879872,66.32156,3,2,1,6
5,BAJAJELEC,BAJAJ ELECTRICALS,Consumer Cyclical,"Furnishings, Fixtures & Appliances",130898919424,93.40164,5,2,2,9
6,BALAMINES,BALAJI AMINES LTD,Basic Materials,Specialty Chemicals,105136414720,30.540884,4,2,2,8
7,MAHABANK,BK OF MAHARASHTRA,Financial Services,Banks—Regional,123841200128,12.602739,3,1,1,5
8,BIRLACORPN,BIRLA CORP,Basic Materials,Building Materials,90569777152,16.869137,5,1,0,6
9,BSOFT,BIRLASOFT (INDIA),Technology,Information Technology Services,126472044544,29.798445,5,2,1,8
